## Fix `feature_selection` plots

In [1]:
import os
import sys
sys.path.append('../')
from model_reboot.EIF_reboot import ExtendedTree, ExtendedIsolationForest,IsolationForest
import numpy as np
import pandas as pd
from utils_reboot.datasets import Dataset
from utils_reboot.utils import *
from utils_reboot.experiments import *
from utils_reboot.plots import *
from sklearn.preprocessing import StandardScaler,MinMaxScaler
import pyod

In [2]:
dataset = Dataset('wine', path = '../data/real/')
dataset.drop_duplicates()
dataset.split_dataset(contamination=0)

In [3]:
os.chdir('../experiments/results')
def fix_feat_sel_plot(dataset,model,interpretation,scenario,save=True,box_loc=None,rotation=False):
    path=os.path.join(os.getcwd(),dataset.name,'experiments','feature_selection',model,interpretation,f'scenario_{str(scenario)}')
    file_path=get_most_recent_file(path)
    path_plots=os.path.join(os.getcwd(),dataset.name,'plots_new')
    plot_feature_selection(file_path,plot_path=path_plots,model=model,interpretation=interpretation,scenario=scenario,save_image=save,plot_image=True,box_loc=box_loc,rotation=rotation)

## Fix `AUC_FS` box

In [1]:
import os
import sys
sys.path.append('../')
from model_reboot.EIF_reboot import ExtendedTree, ExtendedIsolationForest,IsolationForest
import numpy as np
import pandas as pd
from utils_reboot.datasets import Dataset
from utils_reboot.utils import *
from utils_reboot.experiments import *
from utils_reboot.plots import *
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from collections import namedtuple

In [2]:
os.chdir('../experiments/results')
models=['EIF+','EIF','IF']
interpretations=['EXIFFI','DIFFI','RandomForest']
scenarios=[1,2]
dataset_names_syn=['Xaxis','Yaxis','bisect','bisect_3d','bisect_6d']
data_syn_path='/home/davidefrizzo/Desktop/PHD/ExIFFI/data/syn/'
dataset_names_real=['pima','breastw','ionosphere','annthyroid','pendigits','diabetes','shuttle']
data_real_path='/home/davidefrizzo/Desktop/PHD/ExIFFI/data/real/'

Synthetic Datasets

In [3]:
for data in dataset_names_syn:
    dataset = Dataset(data, path = data_syn_path)
    for model in models:
        for interpretation in interpretations:
            if (model=='EIF+' or model=='EIF') and interpretation=='DIFFI':
                continue
            elif (model=='IF') and (interpretation=='EXIFFI'):
                continue
            else:
                for scenario in scenarios:
                    fix_fs_file(dataset,model,interpretation,scenario)

Real Datasets

In [6]:
for data in dataset_names_real:
    dataset = Dataset(data, path = data_real_path)
    for model in models:
        for interpretation in interpretations:
            if (model=='EIF+' or model=='EIF') and interpretation=='DIFFI':
                continue
            elif (model=='IF') and (interpretation=='EXIFFI'):
                continue
            else:
                for scenario in scenarios:
                    fix_fs_file(dataset,model,interpretation,scenario)

IndexError: list index out of range

In [5]:
aucfs_vals=[]
for model in models:
    for interpretation in interpretations:
        if (model=='EIF+' or model=='EIF') and interpretation=='DIFFI':
            continue
        elif (model=='IF') and (interpretation=='EXIFFI'):
            continue
        else:
            for scenario in scenarios:
                aucfs_vals.append(get_fs_file(dataset,model,interpretation,scenario).aucfs)